## Adaptive Wind Driven Optimization submission to the Black Box Optimization Competition
### BBComp: https://bbcomp.ini.rub.de/

<p> BBComp — the black box optimization competition — is the first competition in continuous black-box optimization where test problems are truly black boxes for participants. It is also the first web/online optimization competition in the direct search domain.

This submission is prepared by
- Zikri Bayraktar, Ph.D.
- thewdoalgorithm@gmail.com
- http://www.thewdo.com

For more info and sample codes on AWDO, please visit the homepage!

### If you use this code, please refer to the following two papers in your publications.
1. Z. Bayraktar, M. Komurcu, J. A. Bossard and D. H. Werner, "The Wind Driven Optimization Technique and its Application in Electromagnetics," IEEE Transactions on Antennas and Propagation, Volume 61, Issue 5, pages 2745 - 2757, May 2013.
2. Zikri Bayraktar and Muge Komurcu, "Adaptive Wind Driven Optimization," Proceedings of the 9th EAI International Conference on Bio-Inspired Information and Communications Technologies (formerly BIONETICS), New York City, NY, Dec. 3-5, 2015.

#### Please note that the code is released for educational/academic purposes and it may contain bugs, hence use at your own risk.

In [1]:
# import some modules:
from ctypes import *
from numpy.ctypeslib import ndpointer
import numpy as np
import sys
import platform

import math
import random
import scipy as sp 
import numpy.matlib
from numpy import linalg as LA

In [2]:
# get competition library name:
dllname = ""
if platform.system() == "Windows":
    dllname = "./bbcomp.dll"
elif platform.system() == "Linux":
    dllname = "./libbbcomp.so"
elif platform.system() == "Darwin":
    dllname = "./libbbcomp.dylib"
else:
    sys.exit("unknown platform")
    

# initialize dynamic library
bbcomp = CDLL(dllname)
bbcomp.configure.restype = c_int
bbcomp.login.restype = c_int
bbcomp.numberOfTracks.restype = c_int
bbcomp.trackName.restype = c_char_p
bbcomp.setTrack.restype = c_int
bbcomp.numberOfProblems.restype = c_int
bbcomp.setProblem.restype = c_int
bbcomp.dimension.restype = c_int
bbcomp.numberOfObjectives.restype = c_int
bbcomp.budget.restype = c_int
bbcomp.evaluations.restype = c_int
bbcomp.evaluate.restype = c_int
bbcomp.evaluate.argtypes = [ndpointer(c_double, flags="C_CONTIGUOUS"), ndpointer(c_double, flags="C_CONTIGUOUS")]
bbcomp.history.restype = c_int
bbcomp.history.argtypes = [c_int, ndpointer(c_double, flags="C_CONTIGUOUS"), ndpointer(c_double, flags="C_CONTIGUOUS")]
bbcomp.errorMessage.restype = c_char_p

In [3]:
print("----------------------------------------------")
print("black box example competition client in Python")
print("----------------------------------------------")

# change the track name to trialMO for multi-objective optimization
#track = "trial"
# track = "trialMO"
track = "BBComp2017-1OBJ"

----------------------------------------------
black box example competition client in Python
----------------------------------------------


Replace the keywords << username_here >> and  << password_here >> below to be able to login to the competition servers.

In [4]:
# set configuration options (this is optional)
result = bbcomp.configure(1, "logs/".encode('ascii'))
if result == 0:
    sys.exit("configure() failed: " + str(bbcomp.errorMessage()))

# login with demo account - this should grant access to the "trial" and "trialMO" tracks (for testing and debugging)
result = bbcomp.login("username_here".encode('ascii'), "password_here".encode('ascii'))
if result == 0:
    sys.exit("login() failed: " + str(bbcomp.errorMessage().decode("ascii")))

print("login successful")

login successful


In [5]:
# request the tracks available to this user (this is optional)
numTracks = bbcomp.numberOfTracks()
if numTracks == 0:
    sys.exit("numberOfTracks() failed: " + str(bbcomp.errorMessage().decode("ascii")))

In [6]:
print(str(numTracks) + " track(s):")
for i in range(numTracks):
    trackname = bbcomp.trackName(i).decode("ascii")
    if bool(trackname) == False:
        sys.exit("trackName() failed: " + str(bbcomp.errorMessage().decode("ascii")))

    print("  " + str(i) + ": " + trackname)

7 track(s):
  0: trial
  1: trialMO
  2: BBComp2017-1OBJ
  3: BBComp2017-1OBJ-expensive
  4: BBComp2017-2OBJ
  5: BBComp2017-2OBJ-expensive
  6: BBComp2017-3OBJ


In [7]:
# set the track
result = bbcomp.setTrack(track.encode('ascii'))
if result == 0:
    sys.exit("setTrack() failed: " + str(bbcomp.errorMessage().decode("ascii")))

print("track set to " + track)

track set to BBComp2017-1OBJ


In [8]:
# obtain number of problems in the track
numProblems = bbcomp.numberOfProblems()
if numProblems == 0:
    sys.exit("numberOfProblems() failed: " << str(bbcomp.errorMessage().decode("ascii")))

print("The track consists of " + str(numProblems) + " problems.")

The track consists of 1000 problems.


In [9]:
# For demonstration purposes we optimize only the first problem in the track.
problemID = 0
result = bbcomp.setProblem(problemID)
if result == 0:
    sys.exit("setProblem() failed: " + str(bbcomp.errorMessage().decode("ascii")))

print("Problem ID set to " + str(problemID))

Problem ID set to 0


In [10]:
# obtain problem properties
dim = bbcomp.dimension()
if dim == 0:
    sys.exit("dimension() failed: " + str(bbcomp.errorMessage().decode("ascii")))
print(dim)

2


In [11]:
obj = bbcomp.numberOfObjectives()
if dim == 0:
    sys.exit("numberOfObjectives() failed: " + str(bbcomp.errorMessage().decode("ascii")))

bud = bbcomp.budget()
if bud == 0:
    sys.exit("budget() failed: " + str(bbcomp.errorMessage().decode("ascii")))

evals = bbcomp.evaluations()
if evals < 0:
    sys.exit("evaluations() failed: " + str(bbcomp.errorMessage().decode("ascii")))

In [12]:
print("problem dimension: " + str(dim))
print("number of objectives: " + str(obj))
print("problem budget: " + str(bud))
print("number of already used up evaluations: " + str(evals))

# allocate memory for a search point
point = np.zeros(dim)
value = np.zeros(obj)

problem dimension: 2
number of objectives: 1
problem budget: 400
number of already used up evaluations: 400


In [13]:
def cmaes(counteval, rec, npop, pres, dim): 
    # Refer to purecmaes.m -- https://www.lri.fr/~hansen/purecmaes.m
    # 
    # counteval -- Iteration counter from WDO.
    # rec -- Record of prior values used in CMAES.
    # npop -- number of population members from WDO, each member gets their own set of coefficients determined by the CMAES.
    # pres -- pressure(cost function) computed by WDO for the set of coefficients that CMEAS picked last iteration
    # dim -- number of dimensions of CMAES optimization
    
    if counteval==1:   #Initialization step
        # define 'records' dictionary to keep track the CMAES values over iterations. 
        #print('Init Stage')
        rec['N'] = dim
        rec['xmean'] = np.random.rand(dim,1)
        rec['sigma'] = 0.5
        rec['lambda'] = npop
        rec['mu'] = npop/2
        rec['weights'] = np.log((npop/2)+1/2) - np.log(np.arange(1,np.floor(npop/2)+1))
        rec['mu'] = np.floor(rec['mu']).astype(int)
        rec['weights'] = rec['weights']/np.sum(rec['weights'])
        rec['mueff'] = np.square(np.sum(rec['weights'])) / np.sum( rec['weights'] * rec['weights'])
        rec['cc'] = (4+rec['mueff']/rec['N']) / (rec['N']+4 +2*rec['mueff']/rec['N'])
        rec['cs'] = (rec['mueff']+2) / (rec['N'] + rec['mueff']+5)
        rec['c1'] = 2 / ((np.square(rec['N']+1.3)) + rec['mueff'])
        rec['cmu'] = min(1-rec['c1'], 2*(rec['mueff']-2+1/rec['mueff'])/(np.square(rec['N']+2)+rec['mueff']))
        rec['damps'] = 1 + 2*max(0, np.sqrt((rec['mueff'] -1)/(rec['N']+1))-1) + rec['cs']
        rec['pc'] = np.zeros(dim)
        rec['ps'] = np.zeros(dim)
        rec['B'] = np.eye(dim,dim)
        rec['D'] = np.ones(dim)
        rec['C'] = np.matmul(  np.matmul( rec['B'], np.diag(np.power(rec['D'],2)) ) , rec['B'].T  )
        rec['invsqrtC'] = np.matmul( np.matmul(rec['B'], np.diag(np.power(rec['D'],-1))) , rec['B']) 
        rec['eigeneval'] = 0
        rec['chiN'] = np.power(rec['N'],0.5)* (1-1/(4+rec['N'])+1/(21*np.square(rec['N'])) )
        
        
    #get fitness from WDO pressure
    rec['arfitness'] = pres
    # sort fitness and compute weighted mean into xmean
    arindex = np.argsort(pres)
    rec['arindex'] = arindex
    rec['arfitness'] = np.sort(pres)
    rec['xold'] = rec['xmean']
    mu = rec['mu']
    ridx = arindex[0:mu.astype(int)]
    recarx = np.array(rec['arx'])
    rec['xmean'] = np.matmul(  recarx[:,ridx], rec['weights'] ).reshape(dim,1)            
    
    rec['ps'] = (1-rec['cs']) * rec['ps'] + np.sqrt(rec['cs']*(2-rec['cs'])*rec['mueff'])  * np.matmul(rec['invsqrtC'] , (rec['xmean']-rec['xold'])).T / rec['sigma']
    rec['hsig'] = int( np.sum(rec['ps']*rec['ps']) / (1-np.power((1-rec['cs']),(2*counteval/rec['lambda']))) / rec['N']  <  2+(4/(rec['N']+1))   )
    rec['pc'] = (1-rec['cc']) * rec['pc'] + rec['hsig'] * np.sqrt(rec['cc']*(2-rec['cc'])*rec['mueff']) * (rec['xmean']-rec['xold']).T / rec['sigma']       
    rec['artmp'] = (1/rec['sigma']) * (recarx[:,ridx]) -np.matlib.repmat(rec['xold'],1,rec['mu'])

    rec['C'] = (1-rec['c1']-rec['cmu']) * rec['C'] \
                    + rec['c1'] * (rec['pc'] * rec['pc'].T + (1-rec['hsig']) * rec['cc']*(2-rec['cc']) * rec['C']) \
                    + rec['cmu'] * np.matmul( np.matmul(rec['artmp'] , np.diag(rec['weights'])) , rec['artmp'].T)
    
    rec['sigma'] = rec['sigma']*np.exp( (rec['cs']/rec['damps'])*(LA.norm(rec['ps'])/rec['chiN']-1) )
    
    if (counteval-rec['eigeneval']) >  (rec['lambda'] / (rec['c1']+rec['cmu'])/rec['N']/10):
        rec['eigeneval'] = counteval
        rec['C'] = np.triu(rec['C']) + np.triu(rec['C'],1).T
        rec['D'], rec['B'] = LA.eigh(rec['C'])
        rec['D'] = np.sqrt(rec['D'])
        rec['invsqrtC'] = np.matmul( np.matmul(rec['B'], np.diag( rec['D']**(-1)) )  , rec['B'].T)
        
        
    for k in range(1,rec['lambda']):
        recarx[:,k] = rec['xmean'].T + (rec['sigma']* np.matmul(rec['B'], ((rec['D']*( np.random.standard_normal(size=(rec['N'],1))).T)).reshape(dim,1) )).T
    rec['arx'] = recarx
    
    return rec

In [19]:
# Run AWDO for every single problems in the track:
for nump in range(0, numProblems):
    
    # For demonstration purposes we optimize only the first problem in the track.
    problemID = nump
    result = bbcomp.setProblem(problemID)
    if result == 0:
        sys.exit("setProblem() failed: " + str(bbcomp.errorMessage().decode("ascii")))

    print("Problem ID set to " + str(problemID))
    #-------------------------------------------------------------------

    # obtain problem properties
    dim = bbcomp.dimension()
    if dim == 0:
        sys.exit("dimension() failed: " + str(bbcomp.errorMessage().decode("ascii")))
    print("Dimensions :" +str(dim))
    #--------------------------------------------------------------------

    bud = bbcomp.budget()
    if bud == 0:
        sys.exit("budget() failed: " + str(bbcomp.errorMessage().decode("ascii")))
    print("Problem Budget set to " + str(bud))
    #-------------------------------------------------------------------

#    popsize = dim+np.ceil(10*np.log10(dim)).astype(int)    # population size
    popsize = np.floor(2*dim).astype(int)
    npar = dim        # dimension of the problem
    maxit = np.floor((bud/popsize)).astype(int)     # max number of iterations
    leftover = bud - popsize*maxit  #leftover budget to run at the very end
    
    #print(bud, maxit, popsize, leftover)
    maxV = 0.28      # max allowed speed
    dimMin = 0      # lower dimension boundary
    dimMax = 1      # upper dimension boundary

    rec = dict()
    # randomly initialize the CMAES population
    rec = {'arx': np.random.rand(4,popsize) }
    alp = rec['arx'][0,:]
    g = rec['arx'][1,:]
    c = rec['arx'][2,:]
    RT = rec['arx'][3,:]
    #----------------------------------------------------------

    # Initialize WDO population position, and velocity:
    # Randomize population in the range of [-1,1]:
    pos = 2 * (np.random.rand(popsize, npar) - 0.5)
    vel = maxV * 2 * (np.random.rand(popsize, npar) - 0.5)
    #----------------------------------------------------------

    # Evaluate initial population:
    pres = 10000 * np.ones((popsize))  # initialize the 
    globalpres = 10000
    globalpos = np.zeros(npar)
    a=pres.shape # size of the pres

    for i in range(0, popsize): #pyhon index starts from 0 !!!
        x = ((dimMax-dimMin) * (pos[i,:]+1)/2) + dimMin
        bbcomp.evaluate(x, value)
        pres[i]=value
    
    #----------------------------------------------------------
    # Finding best air parcel in the initial population :

    globalpres = pres.min()         # minimum pressure
    minIndx = np.where(pres == pres.min())  # index of minimum pressure
    globalpos = pos[minIndx,:]       # position vector for the minimum

    minpres = np.zeros(maxit)
    keepglob = np.zeros(maxit)

    indx = np.argsort(pres)      # index of sorted
    pos = pos[indx,:]
    minpres[0] = globalpres       # save the minimum pressure
    keepglob[0] = globalpres;   
    #-----------------------------------------------------------------

    velot = np.zeros((popsize, npar))
    keepglob = np.ones(maxit)
    
    # Start iterations:
    itr = 1 #iteration counter
    for ij in range(1,maxit):
        #update velocity
        for i in range(popsize):
            a = np.random.permutation(range(0,npar)) #random perm    
            velot[i,:] = 1*vel[i,a]
            vel[i,:] = (1-alp[i]) * vel[i,:] - (g[i]*pos[i,:]) + \
                    abs((1/(i+1))-1) *((globalpos-pos[i,:]*RT[i])) + \
                    (c[i]*velot[i,:]/(i+1))
            #python index starts from zero, watch out for division by zero error!

        #check velocity
        vel=vel.clip(-maxV,maxV)
        #update air parcel position
        pos=pos+vel
        pos=pos.clip(-1,1)
        
        #evaluate the new position
        for i in range(0, popsize): #pyhon index starts from 0 !!!
            x = ((dimMax-dimMin) * (pos[i,:]+1)/2) + dimMin
            bbcomp.evaluate(x, value)
            pres[i]=value

        # Finding best air parcel in the initial population :
        mpres = pres.min()         # minimum pressure
        mIndx = np.where(pres == pres.min())  # index of minimum pressure
        gpos = pos[mIndx,:]       # position vector for the minimum

        indx = np.argsort(pres)     # index of sorted
        pos = pos[indx,:]           #sort position
        vel = vel[indx,:]           #sort velocity

        #update inherent parameters through cmaes every X iterations:
        if np.mod(ij, 30) == 1:
            x,y = rec['arx'].shape
            rec = cmaes(ij,rec,popsize,pres,x)
            alp = rec['arx'][0,:]
            g = rec['arx'][1,:]
            c = rec['arx'][2,:]
            RT = rec['arx'][3,:]
    
        if mpres < globalpres:  #if lower pressure found, update the global min
            globalpres = mpres
            globalpos = gpos
            #print(mpres)
        
        keepglob[ij] = globalpres
    
    #Evaluate the leftover budget at the very end:
    for i in range(0, leftover): #pyhon index starts from 0 !!!
        x = ((dimMax-dimMin) * (pos[i,:]+1)/2) + dimMin
        bbcomp.evaluate(x, value)
        pres[i]=value

    #print("Best cost for this problem " + str(globalpres))

Problem ID set to 786
Dimensions :32
Problem Budget set to 102400
Problem ID set to 787
Dimensions :32
Problem Budget set to 102400
Problem ID set to 788
Dimensions :32
Problem Budget set to 102400
Problem ID set to 789
Dimensions :32
Problem Budget set to 102400


In [ ]:
#end-of-file